In [1]:
from google_maps_geocoder.geocoder import GoogleGeocoder
import pandas as pd
import os

# Initialize the GoogleGeocoder
api_key = os.getenv("GOOGLE_API_KEY")
geocoder = GoogleGeocoder(api_key)

# Load the dataset
input_data = pd.read_csv(r"C:\Users\Miles.Vandenburg\Documents\Moderna Portfolio Summary 12.4.2024 (1).csv")

# Clean and prepare the data
destinations, needs_geocoding = geocoder.cleanup_pd(input_data)

# Perform geocoding
if needs_geocoding:
    final_dest_df = geocoder.geocode_addresses(destinations, needs_geocoding)
    final_dest_df.to_csv('geocoded_results.csv', index=False)
else:
    print("Data already contains coordinates.")


Successfully retrieved results for address: 820 Wellington Road,Clayton,VIC
Successfully retrieved results for address: 101 Collins Street,Melbourne,VIC
Successfully retrieved results for address: 211C Wellington Road,Mulgrave,VIC
Successfully retrieved results for address: 405 Bd Armand - Frappier,Laval,QC
Successfully retrieved results for address: 155 Wellington Street West,Toronto,ON
Successfully retrieved results for address: Building 2, No. 288, Xinfu Road, Minhang District,Shanghai,Shanghai
Successfully retrieved results for address: Level 9, Tower 2, China Central Place, No.79 Jianguo Road, Chaoyang District,Beijing,Beijing
Successfully retrieved results for address: 19 Rue Cognacq-Jay,Paris,Ile-de-France
Successfully retrieved results for address: Brienner Strasse 45 a-d,Munich,Bavaria
Successfully retrieved results for address: Via Vittorio Veneto 54b,Rome
Successfully retrieved results for address: 4 Chome-1-1 Toranomon,Tokyo
Successfully retrieved results for address: 2-3-8